In [1]:
import pyspark
import os
import requests

In [2]:
myConf = pyspark.SparkConf()
spark = pyspark.sql.SparkSession\
.builder\
.master('local')\
.appName('myApp')\
.config(conf=myConf)\
.getOrCreate()

21/10/11 03:33:44 WARN Utils: Your hostname, Kritiasui-MacBookAir.local resolves to a loopback address: 127.0.0.1; using 172.30.1.24 instead (on interface en0)
21/10/11 03:33:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/10/11 03:33:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


# URL에서 데이터 읽어오기

In [3]:
r = requests.get('https://raw.githubusercontent.com/jokecamp/FootballData/master/World%20Cups/all-world-cup-players.json')

In [4]:
wc = r.json()

In [11]:
len(wc)

9443

In [5]:
_wcDf = spark.createDataFrame(wc)

- Row를 이용해 데이터프레임을 만드는 것이 좋다.

# Dictionary인자를 풀어서 Row에 넘겨주기

In [6]:
from pyspark.sql import Row
wcDf = spark.createDataFrame(Row(**x) for x in wc)

In [7]:
wcDf.printSchema()

root
 |-- Competition: string (nullable = true)
 |-- Year: long (nullable = true)
 |-- Team: string (nullable = true)
 |-- Number: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- FullName: string (nullable = true)
 |-- Club: string (nullable = true)
 |-- ClubCountry: string (nullable = true)
 |-- DateOfBirth: string (nullable = true)
 |-- IsCaptain: boolean (nullable = true)



In [8]:
wcDf.take(1)

[Row(Competition='World Cup', Year=1930, Team='Argentina', Number='', Position='GK', FullName='Ãngel Bossio', Club='Club AtlÃ©tico Talleres de Remedios de Escalada', ClubCountry='Argentina', DateOfBirth='1905-5-5', IsCaptain=False)]

# RDD 생성

In [12]:
wcRdd = spark.sparkContext.parallelize(wc)

In [13]:
wcRdd.take(1)

21/10/11 03:38:39 WARN TaskSetManager: Stage 3 contains a task of very large size (1044 KiB). The maximum recommended task size is 1000 KiB.


[{'Competition': 'World Cup',
  'Year': 1930,
  'Team': 'Argentina',
  'Number': '',
  'Position': 'GK',
  'FullName': 'Ãngel Bossio',
  'Club': 'Club AtlÃ©tico Talleres de Remedios de Escalada',
  'ClubCountry': 'Argentina',
  'DateOfBirth': '1905-5-5',
  'IsCaptain': False}]

In [14]:
wcDfFromRdd = spark.createDataFrame(wcRdd)
wcDfFromRdd.printSchema()

root
 |-- Club: string (nullable = true)
 |-- ClubCountry: string (nullable = true)
 |-- Competition: string (nullable = true)
 |-- DateOfBirth: string (nullable = true)
 |-- FullName: string (nullable = true)
 |-- IsCaptain: boolean (nullable = true)
 |-- Number: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Team: string (nullable = true)
 |-- Year: long (nullable = true)



21/10/11 03:39:01 WARN TaskSetManager: Stage 4 contains a task of very large size (1044 KiB). The maximum recommended task size is 1000 KiB.


In [15]:
wcDfFromRdd.take(1)

21/10/11 03:39:30 WARN TaskSetManager: Stage 5 contains a task of very large size (1044 KiB). The maximum recommended task size is 1000 KiB.


[Row(Club='Club AtlÃ©tico Talleres de Remedios de Escalada', ClubCountry='Argentina', Competition='World Cup', DateOfBirth='1905-5-5', FullName='Ãngel Bossio', IsCaptain=False, Number='', Position='GK', Team='Argentina', Year=1930)]

# 결측값 | Na
- IsCaptain 항목은 isnan()이 요구하는 float에 맞지 않으므로, 제외한 채로 처리해준다.

In [18]:
cols = wcDf.columns
cols.remove('IsCaptain')

In [34]:
from pyspark.sql.functions import isnan, when, count, col
wcDf.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in cols]).show()
'''
디스플레이: wcDf에 대한 쿼리: cols의 각 c에 대해, c를 레이블로 하며, count를 value로 한다.
count는 각 label에 대해 Null값이나 NaN값을 가진 item에 대해 수행한다.
'''

+-----------+----+----+------+--------+--------+----+-----------+-----------+
|Competition|Year|Team|Number|Position|FullName|Club|ClubCountry|DateOfBirth|
+-----------+----+----+------+--------+--------+----+-----------+-----------+
|          0|   0|   0|     0|       0|       0|   0|          0|          0|
+-----------+----+----+------+--------+--------+----+-----------+-----------+



'\n디스플레이: wcDf에 대한 쿼리: cols의 각 c에 대해, c를 레이블로 하며, count를 value로 한다.\ncount는 각 label에 대해 Null값이나 NaN값을 가진 item에 대해 수행한다.\n'

# 형변환

In [35]:
wcDf.printSchema()

root
 |-- Competition: string (nullable = true)
 |-- Year: long (nullable = true)
 |-- Team: string (nullable = true)
 |-- Number: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- FullName: string (nullable = true)
 |-- Club: string (nullable = true)
 |-- ClubCountry: string (nullable = true)
 |-- DateOfBirth: string (nullable = true)
 |-- IsCaptain: boolean (nullable = true)



DateOfBirth는 String이 아니라 DateTime이어야 한다.

In [37]:
from datetime import datetime
from pyspark.sql.functions import udf
from pyspark.sql.types import DateType
# udf: 사용자 정의 함수. toDate는 함수와 같이 사용할 수 있다.
toDate = udf(lambda x : datetime.strptime(x, '%m/%d/%Y'), DateType())

In [38]:
# withColumn : Df에 Column을 추가하는 함수(컬럼 이름, 아이템)
wcDf = wcDf.withColumn('date1', toDate(wcDf['DateOfBirth']))

In [39]:
wcDf.take(1)

21/10/11 04:03:14 ERROR Executor: Exception in task 0.0 in stage 18.0 (TID 18)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/elplaguister/opt/anaconda3/envs/Analysis/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/Users/elplaguister/opt/anaconda3/envs/Analysis/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/Users/elplaguister/opt/anaconda3/envs/Analysis/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 211, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/Users/elplaguister/opt/anaconda3/envs/Analysis/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 132, in dump_stream
    for obj in iterator:
  File "/Users/elplaguister/opt/anaconda3/envs/Analy

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/Users/elplaguister/opt/anaconda3/envs/Analysis/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/Users/elplaguister/opt/anaconda3/envs/Analysis/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/Users/elplaguister/opt/anaconda3/envs/Analysis/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 211, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/Users/elplaguister/opt/anaconda3/envs/Analysis/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 132, in dump_stream
    for obj in iterator:
  File "/Users/elplaguister/opt/anaconda3/envs/Analysis/lib/python3.8/site

ValueError: time data '1905-5-5' does not match format '%m/%d/%Y'

In [40]:
# drop : Df에 Column을 삭제하는 함수
wcDf = wcDf.drop('date1')

In [42]:
from pyspark.sql.functions import to_date
_wcDfCasted = wcDf.withColumn('date2', to_date(wcDf['DateOfBirth'], 'yyyy-MM-dd'))

In [43]:
from pyspark.sql.types import DateType
wcDfCasted = _wcDfCasted.withColumn('date3', _wcDfCasted['DateOfBirth'].cast(DateType()))
wcDfCasted = wcDfCasted.withColumn('NumberInt', wcDfCasted['Number'].cast('integer'))

In [44]:
wcDfCasted.printSchema()

root
 |-- Competition: string (nullable = true)
 |-- Year: long (nullable = true)
 |-- Team: string (nullable = true)
 |-- Number: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- FullName: string (nullable = true)
 |-- Club: string (nullable = true)
 |-- ClubCountry: string (nullable = true)
 |-- DateOfBirth: string (nullable = true)
 |-- IsCaptain: boolean (nullable = true)
 |-- date2: date (nullable = true)
 |-- date3: date (nullable = true)
 |-- NumberInt: integer (nullable = true)



In [45]:
spark.sql('set spark.sql.legacy.timeParserPolicy=LEGACY')
wcDfCasted.take(1)

[Row(Competition='World Cup', Year=1930, Team='Argentina', Number='', Position='GK', FullName='Ãngel Bossio', Club='Club AtlÃ©tico Talleres de Remedios de Escalada', ClubCountry='Argentina', DateOfBirth='1905-5-5', IsCaptain=False, date2=datetime.date(1905, 5, 5), date3=datetime.date(1905, 5, 5), NumberInt=None)]

In [52]:
%%writefile src/week6_1.py
# import
import os
import requests
import pyspark
from pyspark.sql import Row
from pyspark.sql.types import DateType
# setting spark
myConf = pyspark.SparkConf()
spark = pyspark.sql.SparkSession.builder.master('local').appName('myApp').config(conf = myConf).getOrCreate()
spark.sql('set spark.sql.legacy.timeParserPolicy=LEGACY')
# get json
r = requests.get('https://raw.githubusercontent.com/jokecamp/FootballData/master/World%20Cups/all-world-cup-players.json')
wc = r.json()
wcDf = spark.createDataFrame(Row(**x) for x in wc)
# preprocessing
wcDfCasted = wcDf.withColumn('date3', wcDf['DateOfBirth'].cast(DateType()))
wcDfCasted = wcDfCasted.withColumn('NumberUnt', wcDf['Number'].cast('integer'))

print(wcDfCasted.take(1))

Overwriting src/week6_1.py


In [53]:
!python src/week6_1.py

21/10/11 04:19:16 WARN Utils: Your hostname, Kritiasui-MacBookAir.local resolves to a loopback address: 127.0.0.1; using 172.30.1.24 instead (on interface en0)
21/10/11 04:19:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/10/11 04:19:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/10/11 04:19:18 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
[Row(Competition='World Cup', Year=1930, Team='Argentina', Number='', Position='GK', FullName='Ãngel Bossio', Club='Club AtlÃ©tico Talleres de Remedios de Escalada', ClubCountry='Argentina', DateOfBirth='1905-5-5', IsCaptain=False, date3=datetime.date(1905, 5, 5), NumberUnt=None)]


# Parquet

- Apache Hadoop에서 컬럼 별로 저장하는 데이터 압축형식

In [54]:
wcDf.write.parquet(os.path.join('data', 'wcdf.parquet'))

In [55]:
_getParquet = spark.read.parquet(os.path.join('data', 'wcdf.parquet'))
_getParquet.show()

+-----------+----+---------+------+--------+--------------------+--------------------+-----------+-----------+---------+
|Competition|Year|     Team|Number|Position|            FullName|                Club|ClubCountry|DateOfBirth|IsCaptain|
+-----------+----+---------+------+--------+--------------------+--------------------+-----------+-----------+---------+
|  World Cup|1930|Argentina|      |      GK|        Ãngel Bossio|Club AtlÃ©tico Ta...|  Argentina|   1905-5-5|    false|
|  World Cup|1930|Argentina|      |      GK|        Juan Botasso|Quilmes AtlÃ©tico...|  Argentina| 1908-10-23|    false|
|  World Cup|1930|Argentina|      |      FW|      Roberto Cherro|          Boca Junio|  Argentina|  1907-2-23|    false|
|  World Cup|1930|Argentina|      |      DF|   Alberto Chividini|Central Norte TucumÃ|  Argentina|  1907-2-23|    false|
|  World Cup|1930|Argentina|    10|      FW|                    |Club Atletico Est...|  Argentina|  1909-3-19|    false|
|  World Cup|1930|Argentina|    